---
# Gmsh Tutorials
- this notebook aims to develop my skills using Gmsh Python API
- python-specific reference: https://gmsh.info/doc/texinfo/gmsh.html#x1


In [28]:
# Tutorial 1 - Geometry basics, elementary entities, physical groups
# https://gitlab.onelab.info/gmsh/gmsh/blob/gmsh_4_11_1/tutorials/python/t1.py
import gmsh
import sys

# Before using any functions in the Python API, Gmsh must be initialized:
if not gmsh.is_initialized():
    gmsh.initialize()

# Next we add a new model named "t1":
Name = "test_mesh"
gmsh.model.add(Name)


# 1.POINTS: The first type of `elementary entity' in Gmsh is a `Point'. To create a point
# the Python API function is gmsh.model.geo.addPoint()  or gmsh.model.geo.add_point()
# 3 arguments are the point coordinates (x, y, z)
# - the next (optional) argument is the target mesh size close to the point
# - the last (optional) argument is the point tag/identifier
lc = 1  # target mesh size
z = 0   # fixed z coordinate
gmsh.model.geo.addPoint(0, 0, z, lc, 1)
gmsh.model.geo.addPoint(0, 10, 0, lc, 2)
gmsh.model.geo.addPoint(10, 10, 0, lc, 3)
gmsh.model.geo.addPoint(10, 0, 0, lc, 4)

# 2.LINES: Curves are second type of elementery entities
# Straight line segments: the first 2 arguments are
# - point tags (the start and end points of the line)
# - the last (optional) is the line tag.
gmsh.model.geo.addLine(1, 2, 1)
gmsh.model.geo.addLine(2, 3, 2)
gmsh.model.geo.addLine(3, 4, 3)
gmsh.model.geo.addLine(4, 1, 4)


# 3.SURFACES: to define a simple rectangular surface, curve loop (ordered list of connected curves)
# has to be defined.  Arguments are
# - list of curve tags
# - the curve loop tag (which must be unique
# amongst curve loops) as the second (optional) argument:
gmsh.model.geo.addCurveLoop([1, 2, 3, 4], 1)

# add the surface as a list of curve loops: outline + holes(optional)
gmsh.model.geo.addPlaneSurface([1], 1)


# Before it can be meshed (and, more generally, before it can be used by API
# functions outside of the built-in CAD kernel functions), the CAD entities must
# be synchronized with the Gmsh model
gmsh.model.geo.synchronize()


# physical groups: mathematical ("domain", "boundary"), functional ("left wing", "fuselage") ...
# Here we define a physical curve that groups the left, right and bottom curves
# in a single group (with prescribed tag 5); and a physical surface with name
# "My surface" (with an automatic tag) containing the geometrical surface 1:
gmsh.model.addPhysicalGroup(1, [1, 3, 4], 5)              # curves physical group
gmsh.model.addPhysicalGroup(2, [1], name="My surface")  # surfaces physical group


# generate a 2D mesh
gmsh.model.mesh.generate(2)


# save it to disk (by default only physical groups are saved)
gmsh.write(f"./meshes/{Name}.msh")
gmsh.write(f"./meshes/{Name}.stl")

# By default, if physical groups are defined, Gmsh will export in
# the output mesh file only those elements that belong to at least one physical
# group. To force Gmsh to save all elements, you can use:
#    gmsh.option.setNumber("Mesh.SaveAll", 1)


# visualize the model in the GUI
gmsh.fltk.run()

# when finished with Gmsh Python API:
gmsh.finalize()

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 30%] Meshing curve 2 (Line)
Info    : [ 50%] Meshing curve 3 (Line)
Info    : [ 80%] Meshing curve 4 (Line)
Info    : Done meshing 1D (Wall 0.000488143s, CPU 0.000736s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.00486965s, CPU 0.004591s)
Info    : 143 nodes 288 elements
Info    : Writing './meshes/test_mesh.msh'...
Info    : Done writing './meshes/test_mesh.msh'
Info    : Writing './meshes/test_mesh.stl'...
Info    : Done writing './meshes/test_mesh.stl'
-------------------------------------------------------
Version       : 4.11.1
License       : GNU General Public License
Build OS      : MacOSX-sdk
Build date    : 20230510
Build host    : Mac-1683750338053.local
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blossom Cairo DIntegration Dlopen DomHex Eigen[contrib] Fltk GMP Gmm[contrib] Hxt Jpeg Kbipack MathEx[contrib] Mesh Metis[cont

In [6]:
# Tutorial 2 - Transformations, extruded geometries, volumes

import gmsh
import sys
import numpy as np


# initialize Gmsh
if not gmsh.is_initialized():
    gmsh.initialize()

# add new model
Name = "test_mesh2"
gmsh.model.add(Name)

# setting up the geometry
lc = 1e-2  # target mesh size
gmsh.model.geo.addPoint(0, 0, 0, lc, 1)
gmsh.model.geo.addPoint(.1, 0, 0, lc, 2)
gmsh.model.geo.addPoint(.1, .3, 0, lc, 3)
gmsh.model.geo.addPoint(0, .3, 0, lc, 4)
gmsh.model.geo.addLine(1, 2, 1)
gmsh.model.geo.addLine(3, 2, 2)
gmsh.model.geo.addLine(3, 4, 3)
gmsh.model.geo.addLine(4, 1, 4)
gmsh.model.geo.addCurveLoop([4, 1, -2, 3], 1)
gmsh.model.geo.addPlaneSurface([1], 1)
gmsh.model.geo.synchronize()
gmsh.model.addPhysicalGroup(1, [1, 2, 4], 5)  # left, right and bottom
gmsh.model.addPhysicalGroup(2, [1], name="My surface")  # surface group


#add new points and curves
gmsh.model.geo.addPoint(0, .4, 0, lc, 5)
gmsh.model.geo.addLine(4, 5, 5)


# transformation: translation
gmsh.model.geo.translate([(0, 5)], -0.02, 0, 0)  # translate([(dimension, tag)], dx, dy, dz)

# transformation: rotation
gmsh.model.geo.rotate([(0, 5)], 0, 0.3, 0, 0, 0, 1, -np.pi / 4)  # around (0, 0.3, 0) along the z axis by -Pi/4

# transformation: duplication
ov = gmsh.model.geo.copy([(0, 3)])        # copy([(dim, tag)]) -> returns (dim, tag)
gmsh.model.geo.translate(ov, 0, 0.05, 0)  # translation of the duplicated point


# creating lines to the duplicated point
gmsh.model.geo.addLine(3, ov[0][1], 7)
gmsh.model.geo.addLine(ov[0][1], 5, 8)
gmsh.model.geo.addCurveLoop([5, -8, -7, 3], 10)
gmsh.model.geo.addPlaneSurface([10], 11)

# duplicating and translating surfaces (surface 1 and surface 11)
ov = gmsh.model.geo.copy([(2, 1), (2, 11)])
gmsh.model.geo.translate(ov, 0.12, 0, 0)

print("New surfaces " + str(ov[0][1]) + " and " + str(ov[1][1]))  # printing the new tags

# Creating volumes 1: Manually
gmsh.model.geo.addPoint(0., 0.3, 0.12, lc, 100)
gmsh.model.geo.addPoint(0.1, 0.3, 0.12, lc, 101)
gmsh.model.geo.addPoint(0.1, 0.35, 0.12, lc, 102)

# accessing coordinates of points
gmsh.model.geo.synchronize()         # first synchronize the model
xyz = gmsh.model.getValue(0, 5, [])  # get coordinates of the point 5
gmsh.model.geo.addPoint(xyz[0], xyz[1], 0.12, lc, 103)  # add point above 5

gmsh.model.geo.addLine(4, 100, 110)
gmsh.model.geo.addLine(3, 101, 111)
gmsh.model.geo.addLine(6, 102, 112)
gmsh.model.geo.addLine(5, 103, 113)
gmsh.model.geo.addLine(103, 100, 114)
gmsh.model.geo.addLine(100, 101, 115)
gmsh.model.geo.addLine(101, 102, 116)
gmsh.model.geo.addLine(102, 103, 117)

gmsh.model.geo.addCurveLoop([115, -111, 3, 110], 118)
gmsh.model.geo.addPlaneSurface([118], 119)
gmsh.model.geo.addCurveLoop([111, 116, -112, -7], 120)
gmsh.model.geo.addPlaneSurface([120], 121)
gmsh.model.geo.addCurveLoop([112, 117, -113, -8], 122)
gmsh.model.geo.addPlaneSurface([122], 123)
gmsh.model.geo.addCurveLoop([114, -110, 5, 113], 124)
gmsh.model.geo.addPlaneSurface([124], 125)
gmsh.model.geo.addCurveLoop([115, 116, 117, 114], 126)
gmsh.model.geo.addPlaneSurface([126], 127)

gmsh.model.geo.addSurfaceLoop([127, 119, 121, 123, 125, 11], 128)
gmsh.model.geo.addVolume([128], 129)



# Creating volumes 2: Extruding surface
ov2 = gmsh.model.geo.extrude([ov[1]], 0, 0, 0.12)  # extrude([(dim, tag), vector])


# setting up multiple mesh sizes with setSize()
gmsh.model.geo.mesh.setSize([(0, 103), (0, 105), (0, 109), (0, 102), (0, 28),
                             (0, 24), (0, 6), (0, 5)], lc * 3)


# synchronize with the model
gmsh.model.geo.synchronize()


# creating a physical group
gmsh.model.addPhysicalGroup(3, [129, 130], 1, "The volume")


# generate mesh:
gmsh.model.mesh.generate(3)

# save the mesh:
gmsh.write(f"./meshes/{Name}.msh")
gmsh.write(f"./meshes/{Name}.stl")
# gmsh.write(f"{Name}.geo_unrolled")  # `Gmsh Unrolled GEO' format


# visualize the model in the GUI
gmsh.fltk.run()

# when finished
gmsh.finalize()


New surfaces 12 and 17
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 10%] Meshing curve 2 (Line)
Info    : [ 10%] Meshing curve 3 (Line)
Info    : [ 10%] Meshing curve 4 (Line)
Info    : [ 20%] Meshing curve 5 (Line)
Info    : [ 20%] Meshing curve 7 (Line)
Info    : [ 20%] Meshing curve 8 (Line)
Info    : [ 30%] Meshing curve 13 (Line)
Info    : [ 30%] Meshing curve 14 (Line)
Info    : [ 30%] Meshing curve 15 (Line)
Info    : [ 40%] Meshing curve 16 (Line)
Info    : [ 40%] Meshing curve 18 (Line)
Info    : [ 40%] Meshing curve 19 (Line)
Info    : [ 50%] Meshing curve 20 (Line)
Info    : [ 50%] Meshing curve 110 (Line)
Info    : [ 50%] Meshing curve 111 (Line)
Info    : [ 60%] Meshing curve 112 (Line)
Info    : [ 60%] Meshing curve 113 (Line)
Info    : [ 60%] Meshing curve 114 (Line)
Info    : [ 70%] Meshing curve 115 (Line)
Info    : [ 70%] Meshing curve 116 (Line)
Info    : [ 70%] Meshing curve 117 (Line)
Info    : [ 80%] Meshing curve 131 (Line)
Info    

In [57]:
# Tutorial 3 - Extruded meshes, interactive (ONELAB) parameters, options

import gmsh
import numpy as np
import sys

# initialize Gmsh
if not gmsh.is_initialized():
    gmsh.initialize()

def createGeometryAndMesh(Name):
    # Clear all models and create a new one
    gmsh.clear()
    gmsh.model.add(Name)

    # from tutorial 1
    lc = 0.01
    gmsh.model.geo.addPoint(0, 0, 0, lc, 1)
    gmsh.model.geo.addPoint(.1, 0, 0, lc, 2)
    gmsh.model.geo.addPoint(.1, .3, 0, lc, 3)
    gmsh.model.geo.addPoint(0, .3, 0, lc, 4)
    gmsh.model.geo.addLine(1, 2, 1)
    gmsh.model.geo.addLine(3, 2, 2)
    gmsh.model.geo.addLine(3, 4, 3)
    gmsh.model.geo.addLine(4, 1, 4)
    gmsh.model.geo.addCurveLoop([4, 1, -2, 3], 1)
    gmsh.model.geo.addPlaneSurface([1], 1)
    gmsh.model.geo.synchronize()
    gmsh.model.addPhysicalGroup(1, [1, 2, 4], 5)
    gmsh.model.addPhysicalGroup(2, [1], name="My surface")

    # extrusion along the z axis (keeping the 2D mesh from the XY plane)
    h = 0.1
    ov = gmsh.model.geo.extrude([(2, 1)], # extruding surface (dim 2) with tag 1
                                0, 0, h,  # extrusion vector
                                [8, 2],   # extrude the 2D mesh 2 layers: 8/2 subdivisions
                                [0.5, 1]) # both layers height is h/2



    # revolve: rotational extrusion
    ov = gmsh.model.geo.revolve([(2, 28)],    # rotate surface (dim 2) with tag 28
                                -0.1, 0, 0.1, # centre of rotation
                                0, 1, 0,      # rotation axis Y
                                -np.pi / 2,   # revolve by 90 degrees (only angles < Pi)
                                [7])          # 7 subdivisions


    # twist: rotation + translation
    angle = gmsh.onelab.getNumber('Parameters/Twisting angle')[0]   # ONELAB interactive in GUI
    ov = gmsh.model.geo.twist([(2, 50)],
                              0, 0.15, 0.25,  # rotation centre
                              -2 * h, 0, 0,   # translation
                              1, 0, 0,        # rotational axis
                              angle * np.pi / 180.,  # angle
                              [10],  # 10 subdivisions
                              [],    # ?
                              True)  # recombine or not

    gmsh.model.geo.synchronize()


    # extrusion functions return a vector of extruded entities:
    # "top" of the extruded surface (in `ov[0]'), the created volume (in `ov[1]')
    # and the tags of the side surfaces (in `ov[2]', `ov[3]', ...).

    # grouping all three volumes
    gmsh.model.addPhysicalGroup(3, [1, 2, ov[1][1]], 101)

    gmsh.model.mesh.generate(3)
    gmsh.write(f"./meshes/{Name}.msh")

# options
gmsh.option.setNumber("Geometry.PointNumbers", 1)   # visible point tags
gmsh.option.setColor("Geometry.Color.Points", 255, 165, 0)  # setting the colors
gmsh.option.setColor("General.Color.Text", 255, 255, 255)   # setting the colors
gmsh.option.setColor("Mesh.Color.Points", 255, 0, 0)        # setting the colors

# "Color." can be ommitted in color options:
r, g, b, a = gmsh.option.getColor("Geometry.Points")
gmsh.option.setColor("Geometry.Surfaces", r, g, b, a)

# ONELAB parameters can be modified interactively in the GUI
# defined by JSON-strings https://gitlab.onelab.info/doc/tutorials/-/wikis/ONELAB-JSON-interface
gmsh.onelab.set("""[
  {
    "type":"number",
    "name":"Parameters/Twisting angle",
    "values":[90],
    "min":0,
    "max":120,
    "step":1
  }
]""")

# Create the geometry and mesh it:
createGeometryAndMesh("test_mesh3")

# Launch the GUI and handle the "check" event (recorded in the "ONELAB/Action"
# parameter) to recreate the geometry with a new twisting angle if necessary:
def checkForEvent():
    action = gmsh.onelab.getString("ONELAB/Action")
    if len(action) and action[0] == "check":
        gmsh.onelab.setString("ONELAB/Action", [""])
        createGeometryAndMesh("test_mesh3")
        gmsh.graphics.draw()
    return True


# launch the GUI
gmsh.fltk.initialize()
while gmsh.fltk.isAvailable() and checkForEvent():
    gmsh.fltk.wait()


gmsh.finalize()


Info    : Clearing all models and views...
Info    : Done clearing all models and views
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 10%] Meshing curve 2 (Line)
Info    : [ 10%] Meshing curve 3 (Line)
Info    : [ 20%] Meshing curve 4 (Line)
Info    : [ 20%] Meshing curve 8 (Extruded)
Info    : [ 20%] Meshing curve 9 (Extruded)
Info    : [ 30%] Meshing curve 10 (Extruded)
Info    : [ 30%] Meshing curve 11 (Extruded)
Info    : [ 30%] Meshing curve 13 (Extruded)
Info    : [ 40%] Meshing curve 14 (Extruded)
Info    : [ 40%] Meshing curve 18 (Extruded)
Info    : [ 40%] Meshing curve 22 (Extruded)
Info    : [ 50%] Meshing curve 30 (Extruded)
Info    : [ 50%] Meshing curve 31 (Extruded)
Info    : [ 50%] Meshing curve 32 (Extruded)
Info    : [ 60%] Meshing curve 33 (Extruded)
Info    : [ 60%] Meshing curve 35 (Extruded)
Info    : [ 70%] Meshing curve 36 (Extruded)
Info    : [ 70%] Meshing curve 40 (Extruded)
Info    : [ 70%] Meshing curve 44 (Extruded)
Info    : 

In [9]:
# Tutorial 4 - Holes in surfaces, annotations, entity colors


import gmsh
import math
import sys
import os

# initialize Gmsh
if not gmsh.is_initialized():
    gmsh.initialize()


gmsh.model.add("test_mesh3")

# preparing parameters
cm = 1e-02
e1 = 4.5 * cm
e2 = 6 * cm / 2
e3 = 5 * cm / 2
h1 = 5 * cm
h2 = 10 * cm
h3 = 5 * cm
h4 = 2 * cm
h5 = 4.5 * cm
R1 = 1 * cm
R2 = 1.5 * cm
r = 1 * cm
Lc1 = 0.01     # mesh size 1
Lc2 = 0.003    # mesh size 2

def hypot(a, b):
    return math.sqrt(a * a + b * b)

ccos = (-h5 * R1 + e2 * hypot(h5, hypot(e2, R1))) / (h5 * h5 + e2 * e2)
ssin = math.sqrt(1 - ccos * ccos)


# can redefine a namespace:
gmg = gmsh.model.geo  # gmg ~ gmsh.model.geo

# defining points and tagging them
gmg.addPoint(-e1 - e2, 0, 0, Lc1, 1)
gmg.addPoint(-e1 - e2, h1, 0, Lc1, 2)
gmg.addPoint(-e3 - r, h1, 0, Lc2, 3)
gmg.addPoint(-e3 - r, h1 + r, 0, Lc2, 4)
gmg.addPoint(-e3, h1 + r, 0, Lc2, 5)
gmg.addPoint(-e3, h1 + h2, 0, Lc1, 6)
gmg.addPoint(e3, h1 + h2, 0, Lc1, 7)
gmg.addPoint(e3, h1 + r, 0, Lc2, 8)
gmg.addPoint(e3 + r, h1 + r, 0, Lc2, 9)
gmg.addPoint(e3 + r, h1, 0, Lc2, 10)
gmg.addPoint(e1 + e2, h1, 0, Lc1, 11)
gmg.addPoint(e1 + e2, 0, 0, Lc1, 12)
gmg.addPoint(e2, 0, 0, Lc1, 13)

gmg.addPoint(R1 / ssin, h5 + R1 * ccos, 0, Lc2, 14)
gmg.addPoint(0, h5, 0, Lc2, 15)
gmg.addPoint(-R1 / ssin, h5 + R1 * ccos, 0, Lc2, 16)
gmg.addPoint(-e2, 0.0, 0, Lc1, 17)

gmg.addPoint(-R2, h1 + h3, 0, Lc2, 18)
gmg.addPoint(-R2, h1 + h3 + h4, 0, Lc2, 19)
gmg.addPoint(0, h1 + h3 + h4, 0, Lc2, 20)
gmg.addPoint(R2, h1 + h3 + h4, 0, Lc2, 21)
gmg.addPoint(R2, h1 + h3, 0, Lc2, 22)
gmg.addPoint(0, h1 + h3, 0, Lc2, 23)

gmg.addPoint(0, h1 + h3 + h4 + R2, 0, Lc2, 24)
gmg.addPoint(0, h1 + h3 - R2, 0, Lc2, 25)

# adding lines
gmg.addLine(1, 17, 1)
gmg.addLine(17, 16, 2)
gmg.addLine(14, 13, 4)
gmg.addLine(13, 12, 5)
gmg.addLine(12, 11, 6)
gmg.addLine(11, 10, 7)
gmg.addLine(8, 7, 9)
gmg.addLine(7, 6, 10)
gmg.addLine(6, 5, 11)
gmg.addLine(3, 2, 13)
gmg.addLine(2, 1, 14)
gmg.addLine(18, 19, 15)
gmg.addLine(21, 22, 20)

# other curve primitives than straight lines:
# - splines, B-splines, circle arcs, ellipse arcs
gmg.addCircleArc(14,   # start point tag
                 15,   # circle centre tag
                 16,   # end point
                 3)    # new tag
gmg.addCircleArc(8, 9, 10, 8)
gmg.addCircleArc(3, 4, 5, 12)
gmg.addCircleArc(21, 20, 24, 16)
gmg.addCircleArc(24, 20, 19, 17)
gmg.addCircleArc(18, 23, 25, 18)
gmg.addCircleArc(25, 23, 22, 19)


# We can then define a new surface:
gmg.addCurveLoop([17, -15, 18, 19, -20, 16], 21)
gmg.addPlaneSurface([21], 22)

# define the exterior surface
gmg.addCurveLoop([11, -12, 13, 14, 1, 2, -3, 4, 5, 6, 7, -8, 9, 10], 23)
gmg.addPlaneSurface([23, 21], 24)  # surface with 2 loops (23,21): exterior + hole

# synchronize the model
gmg.synchronize()

# adding some comments by creating a post-processing view
v = gmsh.view.add("comments")

# Add text in *window coordinates*  10 pixels from the left and 10 pixels up
gmsh.view.addListDataString(v, [10, -10], ["This is my string about the mesh"])

# Add text in *model coordinates* centered at (X,Y,Z) = (0, 0.11, 0)
gmsh.view.addListDataString(v, [0, 0.11, 0], ["Hole"],
                            ["Align", "Center", "Font", "Helvetica"])

# 3D annotations with images
png = './meshes/t4_image-to-display.png'
gmsh.view.addListDataString(v, [0, 0.09, 0],   # 3D annotations coords
                            ["file://" + png + "@0.01x0"], # size in model coords
                            ["Align", "Center"])

# The 3D orientation of the image (the direction of B and L edge in model space:
gmsh.view.addListDataString(v, [-0.01, 0.09, 0],
                            ["file://" + png + "@0.01x0,0,0,1,0,1,0"])

# The image in "billboard" mode, always parallel to the camera
gmsh.view.addListDataString(v, [0, 0.12, 0], ["file://" + png + "@0.01x0#"],
                            ["Align", "Center"])

# The size of 2D annotations is given directly in pixels:
gmsh.view.addListDataString(v, [150, -100], ["file://" + png + "@200x0"])

# Views responding to double-click events:
gmsh.view.option.setString(v, "DoubleClickedCommand",
                           "Printf('View[0] has been double-clicked!');")

# geometrical entities responding to double-click events
gmsh.option.setString(
    "Geometry.DoubleClickedLineCommand",
    "Printf('Curve %g has been double-clicked!', "
    "Geometry.DoubleClickedEntityTag);")

# change the color of some entities:
gmsh.model.setColor([(2, 22)], 127, 127, 127)  # Gray50
gmsh.model.setColor([(2, 24)], 160, 32, 240)   # Purple
gmsh.model.setColor([(1, i) for i in range(1, 15)], 255, 0, 0)     # Red
gmsh.model.setColor([(1, i) for i in range(15, 21)], 255, 255, 0)  # Yellow

gmsh.model.mesh.generate(2)

gmsh.write("test_mesh4.msh")

# Launch the GUI
gmsh.fltk.run()

# finish the session
gmsh.finalize()


Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 10%] Meshing curve 2 (Line)
Info    : [ 10%] Meshing curve 3 (Circle)
Info    : [ 20%] Meshing curve 4 (Line)
Info    : [ 20%] Meshing curve 5 (Line)
Info    : [ 30%] Meshing curve 6 (Line)
Info    : [ 30%] Meshing curve 7 (Line)
Info    : [ 40%] Meshing curve 8 (Circle)
Info    : [ 40%] Meshing curve 9 (Line)
Info    : [ 50%] Meshing curve 10 (Line)
Info    : [ 50%] Meshing curve 11 (Line)
Info    : [ 60%] Meshing curve 12 (Circle)
Info    : [ 60%] Meshing curve 13 (Line)
Info    : [ 70%] Meshing curve 14 (Line)
Info    : [ 70%] Meshing curve 15 (Line)
Info    : [ 80%] Meshing curve 16 (Circle)
Info    : [ 80%] Meshing curve 17 (Circle)
Info    : [ 90%] Meshing curve 18 (Circle)
Info    : [ 90%] Meshing curve 19 (Circle)
Info    : [100%] Meshing curve 20 (Line)
Info    : Done meshing 1D (Wall 0.00949972s, CPU 0.009333s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 22 (Plane, Frontal-Delaunay)
Info 

In [31]:
# Tutorial 5 - Mesh sizes, holes in volumes

import gmsh
import math
import sys

# initialize Gmsh
if not gmsh.is_initialized():
    gmsh.initialize()

Name = "test_mesh5"
gmsh.model.add(Name)


# target mesh sizes
lc1 = .1
lc2 = .0005
lc3 = .055

# namespace for gmsh.model.geo
gmg = gmsh.model.geo

# defining a truncated cube:
gmg.addPoint(0.5, 0.5, 0.5, lc2, 1)
gmg.addPoint(0.5, 0.5, 0, lc1, 2)
gmg.addPoint(0, 0.5, 0.5, lc1, 3)
gmg.addPoint(0, 0, 0.5, lc1, 4)
gmg.addPoint(0.5, 0, 0.5, lc1, 5)
gmg.addPoint(0.5, 0, 0, lc1, 6)
gmg.addPoint(0, 0.5, 0, lc1, 7)
gmg.addPoint(0, 1, 0, lc1, 8)
gmg.addPoint(1, 1, 0, lc1, 9)
gmg.addPoint(0, 0, 1, lc1, 10)
gmg.addPoint(0, 1, 1, lc1, 11)
gmg.addPoint(1, 1, 1, lc1, 12)
gmg.addPoint(1, 0, 1, lc1, 13)
gmg.addPoint(1, 0, 0, lc1, 14)

# adding the lines
gmg.addLine(8, 9, 1)
gmg.addLine(9, 12, 2)
gmg.addLine(12, 11, 3)
gmg.addLine(11, 8, 4)
gmg.addLine(9, 14, 5)
gmg.addLine(14, 13, 6)
gmg.addLine(13, 12, 7)
gmg.addLine(11, 10, 8)
gmg.addLine(10, 13, 9)
gmg.addLine(10, 4, 10)
gmg.addLine(4, 5, 11)
gmg.addLine(5, 6, 12)
gmg.addLine(6, 2, 13)
gmg.addLine(2, 1, 14)
gmg.addLine(1, 3, 15)
gmg.addLine(3, 7, 16)
gmg.addLine(7, 2, 17)
gmg.addLine(3, 4, 18)
gmg.addLine(5, 1, 19)
gmg.addLine(7, 8, 20)
gmg.addLine(6, 14, 21)

# adding loops and surfaces
gmg.addCurveLoop([-11, -19, -15, -18], 22)
gmg.addPlaneSurface([22], 23)
gmg.addCurveLoop([16, 17, 14, 15], 24)
gmg.addPlaneSurface([24], 25)
gmg.addCurveLoop([-17, 20, 1, 5, -21, 13], 26)
gmg.addPlaneSurface([26], 27)
gmg.addCurveLoop([-4, -1, -2, -3], 28)
gmg.addPlaneSurface([28], 29)
gmg.addCurveLoop([-7, 2, -5, -6], 30)
gmg.addPlaneSurface([30], 31)
gmg.addCurveLoop([6, -9, 10, 11, 12, 21], 32)
gmg.addPlaneSurface([32], 33)
gmg.addCurveLoop([7, 3, 8, 9], 34)
gmg.addPlaneSurface([34], 35)
gmg.addCurveLoop([-10, 18, -16, -20, 4, -8], 36)
gmg.addPlaneSurface([36], 37)
gmg.addCurveLoop([-14, -13, -12, 19], 38)
gmg.addPlaneSurface([38], 39)

shells = []
sl = gmg.addSurfaceLoop([35, 31, 29, 37, 33, 23, 39, 25, 27])
shells.append(sl)

# This function will create a spherical hole in a volume
def cheeseHole(x, y, z, r, lc, shells):

    p1 = gmsh.model.geo.addPoint(x, y, z, lc)
    p2 = gmsh.model.geo.addPoint(x + r, y, z, lc)
    p3 = gmsh.model.geo.addPoint(x, y + r, z, lc)
    p4 = gmsh.model.geo.addPoint(x, y, z + r, lc)
    p5 = gmsh.model.geo.addPoint(x - r, y, z, lc)
    p6 = gmsh.model.geo.addPoint(x, y - r, z, lc)
    p7 = gmsh.model.geo.addPoint(x, y, z - r, lc)

    c1 = gmsh.model.geo.addCircleArc(p2, p1, p7)    # adding 1/4 arcs, 12x
    c2 = gmsh.model.geo.addCircleArc(p7, p1, p5)
    c3 = gmsh.model.geo.addCircleArc(p5, p1, p4)
    c4 = gmsh.model.geo.addCircleArc(p4, p1, p2)
    c5 = gmsh.model.geo.addCircleArc(p2, p1, p3)
    c6 = gmsh.model.geo.addCircleArc(p3, p1, p5)
    c7 = gmsh.model.geo.addCircleArc(p5, p1, p6)
    c8 = gmsh.model.geo.addCircleArc(p6, p1, p2)
    c9 = gmsh.model.geo.addCircleArc(p7, p1, p3)
    c10 = gmsh.model.geo.addCircleArc(p3, p1, p4)
    c11 = gmsh.model.geo.addCircleArc(p4, p1, p6)
    c12 = gmsh.model.geo.addCircleArc(p6, p1, p7)

    l1 = gmsh.model.geo.addCurveLoop([c5, c10, c4])  # adding 1/8 of the sphere, 8x
    l2 = gmsh.model.geo.addCurveLoop([c9, -c5, c1])
    l3 = gmsh.model.geo.addCurveLoop([c12, -c8, -c1])
    l4 = gmsh.model.geo.addCurveLoop([c8, -c4, c11])
    l5 = gmsh.model.geo.addCurveLoop([-c10, c6, c3])
    l6 = gmsh.model.geo.addCurveLoop([-c11, -c3, c7])
    l7 = gmsh.model.geo.addCurveLoop([-c2, -c7, -c12])
    l8 = gmsh.model.geo.addCurveLoop([-c6, -c9, c2])

    # function addSurfaceFilling()' for non-plane surfaces
    s1 = gmsh.model.geo.addSurfaceFilling([l1])     # 1/8 of sphere 8 surfaces
    s2 = gmsh.model.geo.addSurfaceFilling([l2])
    s3 = gmsh.model.geo.addSurfaceFilling([l3])
    s4 = gmsh.model.geo.addSurfaceFilling([l4])
    s5 = gmsh.model.geo.addSurfaceFilling([l5])
    s6 = gmsh.model.geo.addSurfaceFilling([l6])
    s7 = gmsh.model.geo.addSurfaceFilling([l7])
    s8 = gmsh.model.geo.addSurfaceFilling([l8])

    sl = gmsh.model.geo.addSurfaceLoop([s1, s2, s3, s4, s5, s6, s7, s8])
    v = gmsh.model.geo.addVolume([sl])
    shells.append(sl)
    return v


# We create five holes in the cube:
x = 0
y = 0.75
z = 0
r = 0.09
for t in range(1, 6):
    x += 0.166
    z += 0.166
    v = cheeseHole(x, y, z, r, lc3, shells)
    gmg.addPhysicalGroup(3, [v], t)

# Volume of the cube with holes is defined by 6 surface loops: exterior + 5 holes
gmg.addVolume(shells, 186)

gmg.synchronize()


# physical volume for the hollowed cube
gmsh.model.addPhysicalGroup(3, [186], 10)

#   to only mesh a subset:
# ent = gmsh.model.getEntities()
# gmsh.model.setVisibility(ent, False)
# gmsh.model.setVisibility([(3, 5)], True, True)
# gmsh.option.setNumber("Mesh.MeshOnlyVisible", 1)

# Meshing algorithms can changed globally using options:
gmsh.option.setNumber("Mesh.Algorithm", 6)  # Frontal-Delaunay for 2D meshes
#
# changing mesh algs for individual surfaces
gmsh.model.mesh.setAlgorithm(2, 33, 1)  # settings on surface 1
#
# Optimization: 'Geometric validity of curvilinear finite elements' Journal Comp.Physics
# gmsh.option.setNumber("Mesh.ElementOrder", 2)
# gmsh.option.setNumber("Mesh.HighOrderOptimize", 2)

gmg.synchronize()###
gmsh.model.mesh.generate(3)
gmsh.write(f"./meshes/{Name}.msh")

# Launch the GUI to see the results:
gmsh.fltk.run()

# finish the session
gmsh.finalize()

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 10%] Meshing curve 2 (Line)
Info    : [ 10%] Meshing curve 3 (Line)
Info    : [ 10%] Meshing curve 4 (Line)
Info    : [ 10%] Meshing curve 5 (Line)
Info    : [ 10%] Meshing curve 6 (Line)
Info    : [ 10%] Meshing curve 7 (Line)
Info    : [ 10%] Meshing curve 8 (Line)
Info    : [ 10%] Meshing curve 9 (Line)
Info    : [ 20%] Meshing curve 10 (Line)
Info    : [ 20%] Meshing curve 11 (Line)
Info    : [ 20%] Meshing curve 12 (Line)
Info    : [ 20%] Meshing curve 13 (Line)
Info    : [ 20%] Meshing curve 14 (Line)
Info    : [ 20%] Meshing curve 15 (Line)
Info    : [ 20%] Meshing curve 16 (Line)
Info    : [ 20%] Meshing curve 17 (Line)
Info    : [ 30%] Meshing curve 18 (Line)
Info    : [ 30%] Meshing curve 19 (Line)
Info    : [ 30%] Meshing curve 20 (Line)
Info    : [ 30%] Meshing curve 21 (Line)
Info    : [ 30%] Meshing curve 22 (Circle)
Info    : [ 30%] Meshing curve 23 (Circle)
Info    : [ 30%] Meshing curve 24 (Cir

In [27]:
# Tutorial 6 - Transfinite meshes, deleting entities

import gmsh
import math
import sys

# initialize Gmsh
if not gmsh.is_initialized():
    gmsh.initialize()

Name = "test_mesh6"
gmsh.model.add(Name)

# namespace
gmg = gmsh.model.geo

# Copied from tutorial 1
lc = 1e-2
gmg.addPoint(0, 0, 0, lc, 1)
gmg.addPoint(.1, 0, 0, lc, 2)
gmg.addPoint(.1, .3, 0, lc, 3)
gmg.addPoint(0, .3, 0, lc, 4)
gmg.addLine(1, 2, 1)
gmg.addLine(3, 2, 2)
gmg.addLine(3, 4, 3)
gmg.addLine(4, 1, 4)
gmg.addCurveLoop([4, 1, -2, 3], 1)
gmg.addPlaneSurface([1], 1)

# Delete the surface and the left line, and replace the line with 3 new ones:
gmg.remove([(2, 1)])  # delete type 2 (surface), tag 1
gmg.remove([(1, 4)])  # delete type 1 (line), tag 4
gmg.remove([(2, 1), (1,4)])  # delete tag 1 *and* tag 4


p1 = gmg.addPoint(-0.05, 0.05, 0, lc)
p2 = gmg.addPoint(-0.05, 0.1, 0, lc)
l1 = gmg.addLine(1, p1)
l2 = gmg.addLine(p1, p2)
l3 = gmg.addLine(p2, 4)

# Create surface
gmg.addCurveLoop([2, -1, l1, l2, l3, -3], 2)
gmg.addPlaneSurface([-2], 1)


# ftion 'setTransfiniteCurve()' explicitly specifies location of mesh nodes on the curve.
gmg.mesh.setTransfiniteCurve(2, 20)  # 20 uniformly distributed points
gmg.mesh.setTransfiniteCurve(l1, 6)  # 6 uniform points
gmg.mesh.setTransfiniteCurve(l2, 6)
gmg.mesh.setTransfiniteCurve(l3, 10)

# geometric series location of the mesh
gmg.mesh.setTransfiniteCurve(1, 30, "Progression", -1.2)  # geom. distribution (reversed)
gmg.mesh.setTransfiniteCurve(3, 30, "Progression", 1.2)   # geom. distribution

# meshing the surface
gmg.mesh.setTransfiniteSurface(1, "Left", [1, 2, 3, 4])  # set up the corners of interpolation

# To use quadrilateral mesh elements:
gmg.mesh.setRecombine(2, 1)

# more points
gmg.addPoint(0.2, 0.2, 0, 1.0, 7)
gmg.addPoint(0.2, 0.1, 0, 1.0, 8)
gmg.addPoint(0, 0.3, 0, 1.0, 9)
gmg.addPoint(0.25, 0.2, 0, 1.0, 10)
gmg.addPoint(0.3, 0.1, 0, 1.0, 11)
gmg.addLine(8, 11, 10)
gmg.addLine(11, 10, 11)
gmg.addLine(10, 7, 12)
gmg.addLine(7, 8, 13)
gmg.addCurveLoop([13, 10, 11, 12], 14)
gmg.addPlaneSurface([14], 15)
for i in range(10, 14):
    gmg.mesh.setTransfiniteCurve(i, 10)
#gmg.mesh.setTransfiniteSurface(15)  # no need for corner list if mesh <= 4 corners
gmg.mesh.setTransfiniteSurface(15,
                              "Alternate")  # optional Left/Right/Alternate mesh

gmg.synchronize()

# apply elliptic smoother to the mesh grid:
gmsh.option.setNumber("Mesh.Smoothing", 100)

gmsh.model.mesh.generate(2)
# saving the mesh
gmsh.write(f"./meshes/{Name}.msh")

# Launch the GUI to see the results:
gmsh.fltk.run()

# finish the session
gmsh.finalize()

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 10%] Meshing curve 2 (Line)
Info    : [ 20%] Meshing curve 3 (Line)
Info    : [ 30%] Meshing curve 4 (Line)
Info    : [ 40%] Meshing curve 5 (Line)
Info    : [ 50%] Meshing curve 6 (Line)
Info    : [ 60%] Meshing curve 10 (Line)
Info    : [ 70%] Meshing curve 11 (Line)
Info    : [ 80%] Meshing curve 12 (Line)
Info    : [ 90%] Meshing curve 13 (Line)
Info    : Done meshing 1D (Wall 0.00100911s, CPU 0.001332s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 1 (Transfinite)
Info    : [ 50%] Meshing surface 15 (Transfinite)
Info    : Done meshing 2D (Wall 0.00144778s, CPU 0.001472s)
Info    : 701 nodes 856 elements
Info    : Writing './meshes/test_mesh6.msh'...
Info    : Done writing './meshes/test_mesh6.msh'
-------------------------------------------------------
Version       : 4.11.1
License       : GNU General Public License
Build OS      : MacOSX-sdk
Build date    : 20230510
Build host    : Mac-168375

In [53]:
# Tutorial 7 - Background meshes

import gmsh
import math
import sys

# initialize Gmsh
if not gmsh.is_initialized():
    gmsh.initialize()

# Merge a list-based post-processing view with the mesh sizes (2D function):
gmsh.merge("./meshes/background_mesh.pos")

# add model
Name = "test_mesh7"
gmsh.model.add(Name)

# namespace
gmg = gmsh.model.geo

# Create a simple rectangular geometry:
lc = 1e-2
gmg.addPoint(0, 0, 0, lc, 1)
gmg.addPoint(.1, 0, 0, lc, 2)
gmg.addPoint(.1, .3, 0, lc, 3)
gmg.addPoint(0, .3, 0, lc, 4)
gmg.addLine(1, 2, 1)
gmg.addLine(3, 2, 2)
gmg.addLine(3, 4, 3)
gmg.addLine(4, 1, 4)
gmg.addCurveLoop([4, 1, -2, 3], 1)
gmg.addPlaneSurface([1], 1)
gmg.synchronize()


# Add the post-processing view as a new size field:
bg_field = gmsh.model.mesh.field.add("PostView")
gmsh.model.mesh.field.setNumber(bg_field, "ViewIndex", 0)

# Apply the view as the current background mesh size field:
gmsh.model.mesh.field.setAsBackgroundMesh(bg_field)

# Computing the mesh sizes from the background mesh only (do not use lc params)
gmsh.option.setNumber("Mesh.MeshSizeExtendFromBoundary", 0)
gmsh.option.setNumber("Mesh.MeshSizeFromPoints", 0)
gmsh.option.setNumber("Mesh.MeshSizeFromCurvature", 0)
gmg.synchronize()


# generate the mesh
gmsh.model.mesh.generate(2)

# saving the mesh
gmsh.write(f"./meshes/{Name}.msh")

# Launch the GUI to see the results:
gmsh.fltk.run()

# finish the session
gmsh.finalize()

Info    : Reading './meshes/background_mesh.pos'...
Info    : Done reading './meshes/background_mesh.pos'
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 30%] Meshing curve 2 (Line)
Info    : [ 50%] Meshing curve 3 (Line)
Info    : [ 80%] Meshing curve 4 (Line)
Info    : Done meshing 1D (Wall 0.033759s, CPU 0.033168s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.0669236s, CPU 0.06657s)
Info    : 1788 nodes 3578 elements
Info    : Writing './meshes/test_mesh7.msh'...
Info    : Done writing './meshes/test_mesh7.msh'
-------------------------------------------------------
Version       : 4.11.1
License       : GNU General Public License
Build OS      : MacOSX-sdk
Build date    : 20230510
Build host    : Mac-1683750338053.local
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blossom Cairo DIntegration Dlopen DomHex Eigen[contrib] Fltk GMP Gmm[contrib] Hxt Jpeg Kbipack MathEx[contrib] Mesh

In [1]:
# Tutorial 8 - Post-processing, image export and animations
import gmsh

# initialize Gmsh
if not gmsh.is_initialized():
    gmsh.initialize()

Name = "test_mesh8"
gmsh.model.add(Name)

# namespace
gmg = gmsh.model.geo

# define geometry
lc = 1e-2
gmg.addPoint(0, 0, 0, lc, 1)
gmg.addPoint(.1, 0, 0, lc, 2)
gmg.addPoint(.1, .3, 0, lc, 3)
gmg.addPoint(0, .3, 0, lc, 4)
gmg.addLine(1, 2, 1)
gmg.addLine(3, 2, 2)
gmg.addLine(3, 4, 3)
gmg.addLine(4, 1, 4)
gmg.addCurveLoop([4, 1, -2, 3], 1)
gmg.addPlaneSurface([1], 1)

gmsh.model.geo.synchronize()

# We merge some post-processing views to work on
gmsh.merge('./meshes/view1.pos')
gmsh.merge('./meshes/view1.pos')
gmsh.merge('./meshes/view4.pos')  # contains 2 views inside

# We then set some general options:
gmsh.option.setNumber("General.Trackball", 0)
gmsh.option.setNumber("General.RotationX", 0)
gmsh.option.setNumber("General.RotationY", 0)
gmsh.option.setNumber("General.RotationZ", 0)

white = (255, 255, 255)
black = (0, 0, 0)

# Color options are special
# Setting a color option of "X.Y" actually sets the option "X.Color.Y"
# Sets "General.Color.Background", etc.
gmsh.option.setColor("General.Background", white[0], white[1], white[2])
gmsh.option.setColor("General.Foreground", black[0], black[1], black[2])
gmsh.option.setColor("General.Text", black[0], black[1], black[2])

gmsh.option.setNumber("General.Orthographic", 0) # perspective/orthographic
gmsh.option.setNumber("General.Axes", 1)         # print axis 0/1
gmsh.option.setNumber("General.SmallAxes", 0)

# We also set some options for each post-processing view:

# checking the num of views
v = gmsh.view.getTags()
if len(v) != 4:
    gmsh.logger.write(f"Wrong number of views!{len(v)}", "error")
    gmsh.finalize()
    exit()

# We set some options for each post-processing view:
gmsh.view.option.setNumber(v[0], "IntervalsType", 2)
gmsh.view.option.setNumber(v[0], "OffsetZ", 0.05)
gmsh.view.option.setNumber(v[0], "RaiseZ", 0)
gmsh.view.option.setNumber(v[0], "Light", 1)
gmsh.view.option.setNumber(v[0], "ShowScale", 0)
gmsh.view.option.setNumber(v[0], "SmoothNormals", 1)

gmsh.view.option.setNumber(v[1], "IntervalsType", 1)
# Note that we can't yet set the ColorTable in API
gmsh.view.option.setNumber(v[1], "NbIso", 10)
gmsh.view.option.setNumber(v[1], "ShowScale", 0)

gmsh.view.option.setString(v[2], "Name", "Test plotting")
gmsh.view.option.setNumber(v[2], "Axes", 1)
gmsh.view.option.setNumber(v[2], "IntervalsType", 2)
gmsh.view.option.setNumber(v[2], "Type", 2)
gmsh.view.option.setNumber(v[2], "IntervalsType", 2)
gmsh.view.option.setNumber(v[2], "AutoPosition", 0)
gmsh.view.option.setNumber(v[2], "PositionX", 85)
gmsh.view.option.setNumber(v[2], "PositionY", 50)
gmsh.view.option.setNumber(v[2], "Width", 200)
gmsh.view.option.setNumber(v[2], "Height", 130)
gmsh.view.option.setNumber(v[3], "Visible", 0)




# creating an animation
t = 0  # Initial step

for num in range(1, 4):

    # Set time step
    for vv in v:
        gmsh.view.option.setNumber(vv, "TimeStep", t)
    current_step = gmsh.view.option.getNumber(v[0], "TimeStep")
    max_step = gmsh.view.option.getNumber(v[0], "NbTimeStep") - 1
    if current_step < max_step:
        t = t + 1
    else:
        t = 0

    gmsh.view.option.setNumber(v[0], "RaiseZ",
                               gmsh.view.option.getNumber(v[0], "RaiseZ") +
                               0.01 / gmsh.view.option.getNumber(v[0], "Max") * t)

    if num == 3:
        # Resize the graphics when num == 3, to create 640x480 frames
        gmsh.option.setNumber("General.GraphicsWidth",
                              gmsh.option.getNumber("General.MenuWidth") + 640)
        gmsh.option.setNumber("General.GraphicsHeight", 480)

    frames = 50
    for num2 in range(frames):
        # Incrementally rotate the scene
        gmsh.option.setNumber("General.RotationX",
                              gmsh.option.getNumber("General.RotationX") + 10)
        gmsh.option.setNumber("General.RotationY",
                              gmsh.option.getNumber("General.RotationX") / 3)
        gmsh.option.setNumber("General.RotationZ",
                              gmsh.option.getNumber("General.RotationZ") + 0.1)

        # Draw the scene
        gmsh.graphics.draw()

        if num == 3:
            # Uncomment the following lines to save each frame to an image file
            #gmsh.write("t8-{}.gif".format(num2))
            #gmsh.write("t8-{}.ppm".format(num2))
            #gmsh.write("t8-{}.jpg".format(num2))
            pass

    if num == 3:
        # Here we could make a system call to generate a movie with ffmeg:
        # import subprocess
        # subprocess.call("ffmpeg -i t8-%d.jpg t8.mpg".split(' '))
        pass



# saving the mesh
gmsh.write(f"./meshes/{Name}.msh")

# Launch the GUI to see the results:
gmsh.fltk.run()

# finish the session
gmsh.finalize()

Info    : Reading './meshes/view1.pos'...
Info    : Done reading './meshes/view1.pos'
Info    : Reading './meshes/view1.pos'...
Info    : Done reading './meshes/view1.pos'
Info    : Reading './meshes/view4.pos'...
Info    : Done reading './meshes/view4.pos'
Info    : Writing './meshes/test_mesh8.msh'...
Info    : Done writing './meshes/test_mesh8.msh'
-------------------------------------------------------
Version       : 4.11.1
License       : GNU General Public License
Build OS      : MacOSX-sdk
Build date    : 20230510
Build host    : Mac-1683750338053.local
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blossom Cairo DIntegration Dlopen DomHex Eigen[contrib] Fltk GMP Gmm[contrib] Hxt Jpeg Kbipack MathEx[contrib] Mesh Metis[contrib] Mpeg Netgen ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR Voro++[contrib] WinslowUntangler Zlib
FLTK version  : 1.3.8
OCC version   : 7.7.1
Packaged by  

In [43]:
# Tutorial 9 - Plugins
import gmsh

# initialize Gmsh
if not gmsh.is_initialized():
    gmsh.initialize()

Name = "test_mesh9"
gmsh.model.add(Name)

# loading a view
gmsh.merge('./meshes/view3.pos')
v = gmsh.view.getTags()
if len(v) != 1:
    gmsh.logger.write("Wrong number of views!", "error")
    gmsh.finalize()
    exit()

# Isosurface' plugin
gmsh.plugin.setNumber("Isosurface", "Value", 0.67)
gmsh.plugin.setNumber("Isosurface", "View", 0)
v1 = gmsh.plugin.run("Isosurface")


# `CutPlane' plugin (plane A*x+B*y+C*z+D=0)
gmsh.plugin.setNumber("CutPlane", "A", 0)
gmsh.plugin.setNumber("CutPlane", "B", 0.2)
gmsh.plugin.setNumber("CutPlane", "C", 1)
gmsh.plugin.setNumber("CutPlane", "D", 0)
gmsh.plugin.setNumber("CutPlane", "View", 0)
v2 = gmsh.plugin.run("CutPlane")

# Add a title (window coords greater than 99999 represents the center)
gmsh.plugin.setString("Annotate", "Text", "A nice little title")
gmsh.plugin.setNumber("Annotate", "X", 1.e5)
gmsh.plugin.setNumber("Annotate", "Y", 50)
gmsh.plugin.setString("Annotate", "Font", "Times-BoldItalic")
gmsh.plugin.setNumber("Annotate", "FontSize", 28)
gmsh.plugin.setString("Annotate", "Align", "Center")
gmsh.plugin.setNumber("Annotate", "View", 0)
gmsh.plugin.run("Annotate")

gmsh.plugin.setString("Annotate", "Text", "(and another little subtitle)")
gmsh.plugin.setNumber("Annotate", "Y", 70)
gmsh.plugin.setString("Annotate", "Font", "Times-Roman")
gmsh.plugin.setNumber("Annotate", "FontSize", 12)
gmsh.plugin.run("Annotate")


# setting options:
gmsh.view.option.setNumber(v[0], "Light", 1)
gmsh.view.option.setNumber(v[0], "IntervalsType", 0)
gmsh.view.option.setNumber(v[0], "NbIso", 10)
gmsh.view.option.setNumber(v[0], "SmoothNormals", 1)
gmsh.view.option.setNumber(v1, "IntervalsType", 2)
gmsh.view.option.setNumber(v2, "IntervalsType", 2)


# saving the mesh
gmsh.write(f"./meshes/{Name}.msh")

# Launch the GUI to see the results:
gmsh.fltk.run()

# finish the session
gmsh.finalize()

Info    : Reading './meshes/view3.pos'...
Info    : Done reading './meshes/view3.pos'
Info    : Running Plugin(Isosurface)...
Info    : Done running Plugin(Isosurface)
Info    : Running Plugin(CutPlane)...
Info    : Done running Plugin(CutPlane)
Info    : Running Plugin(Annotate)...
Info    : Done running Plugin(Annotate)
Info    : Running Plugin(Annotate)...
Info    : Done running Plugin(Annotate)
Info    : Writing './meshes/test_mesh9.msh'...
Info    : Done writing './meshes/test_mesh9.msh'
-------------------------------------------------------
Version       : 4.11.1
License       : GNU General Public License
Build OS      : MacOSX-sdk
Build date    : 20230510
Build host    : Mac-1683750338053.local
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blossom Cairo DIntegration Dlopen DomHex Eigen[contrib] Fltk GMP Gmm[contrib] Hxt Jpeg Kbipack MathEx[contrib] Mesh Metis[contrib] Mpeg Netgen ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom Parser Plugins Pn

In [14]:
# Tutorial 10 - Mesh size fields
import gmsh

# initialize Gmsh
if not gmsh.is_initialized():
    gmsh.initialize()

Name = "test_mesh10"
gmsh.model.add(Name)

# namespace
gmg = gmsh.model.geo

# adding geometry
# Let's create a simple rectangular geometry:
lc = .15
gmg.addPoint(0.0, 0.0, 0, lc, 1)
gmg.addPoint(1, 0.0, 0, lc, 2)
gmg.addPoint(1, 1, 0, lc, 3)
gmg.addPoint(0, 1, 0, lc, 4)
gmg.addPoint(0.2, .5, 0, lc, 5)

gmg.addLine(1, 2, 1)
gmg.addLine(2, 3, 2)
gmg.addLine(3, 4, 3)
gmg.addLine(4, 1, 4)

gmg.addCurveLoop([1, 2, 3, 4], 5)
gmg.addPlaneSurface([5], 6)
gmg.synchronize()


# we want mesh elements with size lc/30 near curve 2 and point 5, and lc elsewhere
# To achieve this, we use mesh size fields: "Distance" and "Threshold"
gmsh.model.mesh.field.add("Distance", 1)  # create a distance field
gmsh.model.mesh.field.setNumbers(1, "PointsList", [5]) # for point 5
gmsh.model.mesh.field.setNumbers(1, "CurvesList", [2]) # and curve 2
gmsh.model.mesh.field.setNumber(1, "Sampling", 100)    # sample curve 2 at 100 points

# We then define a `Threshold' field, which uses the return value of the
# `Distance' field 1 in order to define a simple change in element size
# SizeMax -                     /------------------
#                              /
#                             /
#                            /
# SizeMin -o----------------/
#          |                |    |
#        Point         DistMin  DistMax
gmsh.model.mesh.field.add("Threshold", 2)
gmsh.model.mesh.field.setNumber(2, "InField", 1)
gmsh.model.mesh.field.setNumber(2, "SizeMin", lc / 30)
gmsh.model.mesh.field.setNumber(2, "SizeMax", lc)
gmsh.model.mesh.field.setNumber(2, "DistMin", 0.15)
gmsh.model.mesh.field.setNumber(2, "DistMax", 0.5)


# Say we want to modulate the mesh element sizes using a mathematical function
# of the spatial coordinates. We can do this with the MathEval field:
gmsh.model.mesh.field.add("MathEval", 3)
gmsh.model.mesh.field.setString(3, "F",
                                "Cos(4*3.14*x) * Sin(4*3.14*y) / 10 + 0.101")

# `Distance' field around point 1 (combine with MathEval field)
gmsh.model.mesh.field.add("Distance", 4)
gmsh.model.mesh.field.setNumbers(4, "PointsList", [1])

# create a 'MathEval' field 5 depends on field 4 (cubic distance to point 1)
gmsh.model.mesh.field.add("MathEval", 5)
gmsh.model.mesh.field.setString(5, "F", "F4^3 + " + str(lc / 100))

# use a `Box' field to impose a step change in element sizes in the box
gmsh.model.mesh.field.add("Box", 6)
gmsh.model.mesh.field.setNumber(6, "VIn", lc / 15)
gmsh.model.mesh.field.setNumber(6, "VOut", lc)
gmsh.model.mesh.field.setNumber(6, "XMin", 0.3)
gmsh.model.mesh.field.setNumber(6, "XMax", 0.6)
gmsh.model.mesh.field.setNumber(6, "YMin", 0.3)
gmsh.model.mesh.field.setNumber(6, "YMax", 0.6)
gmsh.model.mesh.field.setNumber(6, "Thickness", 0.3)

# Let's use the minimum of all the fields as the mesh size field:
gmsh.model.mesh.field.add("Min", 7)   # minimum of all fields
gmsh.model.mesh.field.setNumbers(7, "FieldsList", [2, 3, 5, 6])
gmsh.model.mesh.field.setAsBackgroundMesh(7)

# The API allows to set a global mesh size callback
def meshSizeCallback(dim, tag, x, y, z, lc):
    return min(lc, 0.02 * x + 0.01)
gmsh.model.mesh.setSizeCallback(meshSizeCallback)


# To determine the size of mesh elements, Gmsh locally computes the minimum of:
#   1) the size of the model bounding box;
#   2) if `Mesh.MeshSizeFromPoints' is set, the mesh size specified at geometrical
#      points;
#   3) if `Mesh.MeshSizeFromCurvature' is positive, the mesh size based on
#      curvature (the value specifying the number of elements per 2 * pi rad);
#   4) the background mesh size field;
#   5) any per-entity mesh size constraint;


# When the element size is fully specified by a mesh size field, we can set:
gmsh.option.setNumber("Mesh.MeshSizeExtendFromBoundary", 0)
gmsh.option.setNumber("Mesh.MeshSizeFromPoints", 0)
gmsh.option.setNumber("Mesh.MeshSizeFromCurvature", 0)
gmsh.model.setColor([(2, 6)], 255, 0, 0)  # Red
gmg.synchronize()

# "Frontal-Delaunay" (6) 2D meshing algorithm -> highest quality meshes
# "Delaunay" (5) 2D meshing algorithm -> handles complex mesh size fields better
gmsh.option.setNumber("Mesh.Algorithm", 5)  # setting the meshing algorithm

gmsh.model.mesh.generate(2)

# saving the mesh
gmsh.write(f"./meshes/{Name}.msh")

# Launch the GUI to see the results:
gmsh.fltk.run()

# finish the session
gmsh.finalize()

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 30%] Meshing curve 2 (Line)
Info    : [ 50%] Meshing curve 3 (Line)
Info    : [ 80%] Meshing curve 4 (Line)
Info    : Done meshing 1D (Wall 0.136817s, CPU 0.136003s)
Info    : Meshing 2D...
Info    : Meshing surface 6 (Plane, Delaunay)
Info    : Done meshing 2D (Wall 1.04521s, CPU 1.03404s)
Info    : 25111 nodes 50223 elements
Info    : Writing './meshes/test_mesh10.msh'...
Info    : Done writing './meshes/test_mesh10.msh'
-------------------------------------------------------
Version       : 4.11.1
License       : GNU General Public License
Build OS      : MacOSX-sdk
Build date    : 20230510
Build host    : Mac-1683750338053.local
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blossom Cairo DIntegration Dlopen DomHex Eigen[contrib] Fltk GMP Gmm[contrib] Hxt Jpeg Kbipack MathEx[contrib] Mesh Metis[contrib] Mpeg Netgen ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom Parser Plugi

In [6]:
gmsh.finalize()

Error   : Gmsh has not been initialized
Error   : Gmsh has not been initialized


Exception: Could not get last error

In [ ]:
# Tutorial -

# initialize Gmsh
if not gmsh.is_initialized():
    gmsh.initialize()

Name = "test_mesh6"
gmsh.model.add(Name)

# namespace
gmg = gmsh.model.geo


# saving the mesh
gmsh.write(f"./meshes/{Name}.msh")

# Launch the GUI to see the results:
gmsh.fltk.run()

# finish the session
gmsh.finalize()